In [1]:
import numpy as np
from model.utils import pytorch_cos_sim
from data.dataloader import convert_to_tensor, example_model_setting
import json
import glob
import xml.etree.ElementTree as et

/home/piai/miniconda3/envs/boaz/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-19 09:53:36.428497: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def preprocess(paths):
	corpus = []
	valid_idx = []
	for i, path in enumerate(paths):
		try:
			tree = et.parse(path)
			root = tree.getroot()
			txt = root[0][2].text.replace('\n', '').replace('\t', '')
			corpus.append(' '.join(txt.split()))
			valid_idx.append(i)
		except:
			print(path, "can't open")
	return corpus, valid_idx

In [3]:
train_data_path = glob.glob('../data/training/data/01_verdict/*/*.xml')
train_label_path = glob.glob('../data/training/label/01_verdict/*/*.json')

train_corpus, valid_idx = preprocess(train_data_path)
train_label_path = [train_label_path[i] for i in valid_idx]

../data/training/data/01_verdict/01_civil/2017나10869.xml can't open
../data/training/data/01_verdict/01_civil/2017나111176.xml can't open
../data/training/data/01_verdict/01_civil/2017나11160.xml can't open
../data/training/data/01_verdict/01_civil/2017나115369.xml can't open
../data/training/data/01_verdict/01_civil/2017나15475.xml can't open
../data/training/data/01_verdict/01_civil/2017나2002746.xml can't open
../data/training/data/01_verdict/02_criminal/2017노2053.xml can't open
../data/training/data/01_verdict/02_criminal/2017노2465.xml can't open
../data/training/data/01_verdict/03_administration/2017누63483.xml can't open
../data/training/data/01_verdict/03_administration/2017누7307.xml can't open
../data/training/data/01_verdict/03_administration/2017누7918.xml can't open
../data/training/data/01_verdict/03_administration/2018구합224.xml can't open
../data/training/data/01_verdict/03_administration/2018구합56077.xml can't open


In [4]:
train_corpus[0][:1000]

'대구고법 2018. 7. 20. 선고 2016나272 판결 : 확정대구고법 2018. 7. 20.선고 2016나272 판결 : 확정 계약체결절차이행 , 각공2018하,182 판시사항 갑 아파트 입주자대표회의가 장기수선충당금으로 실시하는 아파트 건물 균열보수 등 공사의 수급인을 선정하기 위하여 국토교통부장관이 구 주택법 제45조의7 에 의하여 운영하는 공동주택관리정보시스템을 통한 전자입찰을 공고하고 현장설명회에서 지하주차장 천장균열 누수부는 반드시 지하주차장특허공법을 사용하여야 한다 고 기재된 시방서를 배부하자, 을 000가 위 특허공법에 대한 사용승인서를 제출하여 입찰하였고, 개찰을 위한 입주자대표회의에서 갑 아파트 관리소장이 입주자대표회의 회장이 개찰에 반대하며 회의장을 나간 상태에서 동대표 병의 지시에 따라 공동주택관리정보시스템에 접속하여 위 사용승인서를 제출한 업체들을 서류심사 통과업체로 선정한 다음 개찰을 실시하여 위 업체들 중 입찰가액이 최저인 을 회사가 낙찰자로 결정되었다고 입력함으로써 이러한 내용의 전자문서가 위 시스템에 게시되었는데, 그 후 갑 아파트 입주자대표회의가을 회사에 위 입찰에서 을 회사가 낙찰되었지만 입찰과정에서 구 주택관리업자 및 사업자 선정지침 등을 위반하였으므로 낙찰이 취소되었다 는 취지의 문서를 보내자, 을 000가 계약체결의 이행 등을 구한 사안에서,을 000의 입찰은 위 선정지침 제5조 별표 3 제4호를 위반하여 무효라고 한 사례 판결요지 갑 아파트 입주자대표회의가 장기수선충당금으로 실시하는 아파트 건물 균열보수 등 공사의 수급인을 선정하기 위하여 국토교통부장관이 구 주택법(2014. 12. 31. 법률 제12959호로 개정되기 전의 것, 이하 구 주택법 이라 한다) 제45조의7 에 의하여 운영하는 공동주택관리정보시스템을 통한 전자입찰을 공고하고 현장설명회에서 지하주차장 천장균열 누수부는 반드시 지하주차장특허공법을 사용하여야 한다 고 기재된 시방서를 배부하자, 을 000가 위 특허공법에 대한 사용승인서를 제출하여 입찰하였고

In [5]:
test_data_path = glob.glob('../data/validation/data/01_verdict/*/*.xml')
test_label_path = glob.glob('../data/validation/label/01_verdict/*/*.json')

test_corpus, valid_idx = preprocess(test_data_path)
test_label_path = [test_label_path[i] for i in valid_idx]

../data/validation/data/01_verdict/02_criminal/2017고단6135.xml can't open


In [6]:
model_ckpt = 'output/nli_checkpoint.pt'
model, transform, device = example_model_setting(model_ckpt)

inputs_corpus = convert_to_tensor(train_corpus, transform)
corpus_embeddings = model.encode(inputs_corpus, device)

using cached model. /home/piai/workspace/boaz_miniproject/KoSimCSE/.cache/kobert_v1.zip
using cached model. /home/piai/workspace/boaz_miniproject/KoSimCSE/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/piai/workspace/boaz_miniproject/KoSimCSE/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/home/piai/workspace/boaz_miniproject/KoSimCSE/data/dataloader.py:178: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  inputs = {'source': torch.LongTensor(tensor_corpus),


In [8]:
test_num = 2
queries = [test_corpus[test_num]]

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
top_k_idx = []
for query in queries:
    query_embedding = model.encode(convert_to_tensor([query], transform), device)
    cos_scores = pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu().detach().numpy()

    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("Query:", query[:200])
    print("\n\n======================\n\n")
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print('idx:',idx)
        top_k_idx.append(idx)
        print(train_corpus[idx].strip()[:200], "(Score: %.4f)" % (cos_scores[idx]))

Query: 서울중앙지방법원 2018. 10. 26. 선고 2014가합530889 판결1서울중앙지방법원 2018. 10. 26. 고 2014가합530889 결손해배상(기), 미간행전문원고 대한민국 (소송대리인 000 담당변호사 00)피고 000 외 1인 (소송대리인 법무법인 00 외 1인)주문1. 원고의 피고 000에 대한 소를 각하한다.2. 피고 000 주식회사는 원





Top 5 most similar sentences in corpus:
idx: 116
서울고등법원 2018. 10. 26. 선고 2018나2025678 판결1서울고등법원 2018. 10. 26.선고 2018나2025678 판결장비임대료청구등, 미간행전문원고, 피항소인 겸 항소인 000 (소송대리인 법무법인 000 담당변호사 00) 피고, 항소인 겸 피항소인 000 (소송대리인 변호사 00)대상판결제1심판결 서울중앙지방법원 2018. 5. 1 (Score: 0.8666)
idx: 67
서울중앙지방법원 2018. 9. 14. 선고 2017나48279 판결서울중앙지방법원 2018. 9. 14.선고 2017나48279 판결 손해배상(자) , 미간행 전문 원고, 피항소인 원고 1 외 4인 (소송대리인 변호사 00) 피고, 항소인 000 (소송대리인 000 담당변호사 00) 대상판결 제1심판결 서울중앙지방법원 2017. 6. 28. 선고 2015 (Score: 0.8580)
idx: 30
서울중앙지방법원 2018. 3. 14. 선고 2017가합549280 제36민사부 판결추심금사 건2017가합549280 추심금원 고000피 고000변 론 종 결2018. 2. 28.판 결 선 고2018. 3. 14.주 문1. 원고의 청구를 기각한다.2. 소송비용은 원고가 부담한다.청 구 취 지피고는 원고에게 5,645,837,818원 및 이에 대하여 2017 (Score: 0.8314)
idx: 24
부산지방법원 2018. 10. 10. 선고 2017가합2086 판결부산지방법원 2018. 10. 10.선고 201

In [9]:
with open(test_label_path[test_num], 'r') as f:
	test_label = json.load(f)
test_label['info']['relateLaword']

['중재법 제9조 ',
 '중재법 제2조 ',
 '민법 제667조 ',
 '구 군수품관리법 제24조',
 '민법 제391조',
 '국가재정법 제96조 ',
 '민법 제166조 ']

In [10]:
for idx in top_k_idx:
	with open(train_label_path[idx], 'r') as f:
		train_label = json.load(f)
	print("idx:",idx)
	print(train_label['info']['relateLaword'])
	print()

idx: 116
['특허법 제126조', '민사소송법 제1조', '민사소송법 제287조', '민사소송규칙 제47조', '민사소송규칙 제70조', '민사소송규칙 제82조', '민사소송법 제146조', '민사소송법 제272조', '민사소송법 제147조', '민사소송법 제285조', '민사소송법 제277조', '민사소송법 제149조']

idx: 67
['민사소송법 제420조', '채무자 회생법 제179조 ']

idx: 30
['전자상거래 등에서의 소비자보호에 관한 법률 제13조', '전자상거래 등에서의 소비자보호에 관한 법률 제17조', '전자상거래 등에서의 소비자보호에 관한 법률 제2항', '전자상거래 등에서의 소비자보호에 관한 법률 제18조', '전자상거래 등에서의 소비자보호에 관한 법률 제9항', ' 전자상거래 등에서의 소비자보호에 관한 법률 제11항', '전자상거래 등에서의 소비자보호에 관한 법률 제35조']

idx: 24
['채무자 회생 및 파산에 관한 법률 제250조', '기술보증기금법 제37조', '신용보증기금법 제30조']

idx: 62
['민법 제741조', '지방세법 시행령 제20조', '지방세기본법 제35조', '지방세기본법 제53조', '지방세법 제7조', '지방세법 제18조', '지방세법 제20조', '지방세법 제21조', '지방세법 제150조', '지방세법 제152조']

